# 🌍 AQI Prediction Analysis## Phân tích và Dự đoán Chỉ số Chất lượng Không khí (AQI)Notebook này thực hiện phân tích toàn diện về dữ liệu AQI và xây dựng các models Machine Learning để dự đoán chỉ số AQI cho 24 giờ tiếp theo.### Objectives:1. Exploratory Data Analysis (EDA)2. Feature Engineering3. Model Training (Random Forest, XGBoost, LSTM)4. Model Evaluation và Comparison5. 24-Hour Prediction---

## 1. Import Libraries và Setup

In [ ]:
# Import librariesimport pandas as pdimport numpy as npimport matplotlib.pyplot as pltimport seaborn as snsfrom pathlib import Pathimport warningswarnings.filterwarnings('ignore')# Import custom modulesimport syssys.path.append('..')from src.data_preprocessing import preprocess_data, check_data_qualityfrom src.feature_engineering import engineer_featuresfrom src.model_training import AQIModelTrainerfrom src.model_evaluation import calculate_metrics, compare_models, plot_predictionsfrom src.prediction import predict_next_24h, visualize_predictions# Setup plottingsns.set_style('whitegrid')plt.rcParams['figure.dpi'] = 100plt.rcParams['figure.figsize'] = (15, 6)print("✅ All libraries imported successfully!")

## 2. Data Loading và Exploration### 2.1 Load Data

In [ ]:
# Load datadata_path = '../data/sample_data.csv'df = preprocess_data(data_path)print(f"\nDataset shape: {df.shape}")print(f"Date range: {df['datetime'].min()} to {df['datetime'].max()}")print(f"\nFirst few rows:")df.head()

### 2.2 Basic Statistics

In [ ]:
# Statistical summaryprint("Statistical Summary:")print("="*70)df.describe()

### 2.3 Missing Values Check

In [ ]:
# Check missing valuesmissing = df.isnull().sum()print("Missing Values:")print(missing[missing > 0] if missing.sum() > 0 else "✅ No missing values!")# Data quality reportquality_report = check_data_quality(df)

### 2.4 Target Variable Distribution

In [ ]:
# AQI distributionfig, axes = plt.subplots(1, 2, figsize=(15, 5))# Bar plotaqi_counts = df['aqi'].value_counts().sort_index()axes[0].bar(aqi_counts.index, aqi_counts.values, color='steelblue', alpha=0.7)axes[0].set_xlabel('AQI Level', fontsize=12)axes[0].set_ylabel('Count', fontsize=12)axes[0].set_title('AQI Distribution', fontsize=14, fontweight='bold')axes[0].grid(True, alpha=0.3)# Pie chartcolors = ['#2ecc71', '#f39c12', '#e74c3c', '#c0392b', '#8e44ad']axes[1].pie(aqi_counts.values, labels=aqi_counts.index, autopct='%1.1f%%',            colors=colors[:len(aqi_counts)], startangle=90)axes[1].set_title('AQI Distribution (%)', fontsize=14, fontweight='bold')plt.tight_layout()plt.show()print(f"\nAQI Statistics:")print(df['aqi'].value_counts().sort_index())

## 3. Feature Analysis### 3.1 Time Series Plots for Pollutants

In [ ]:
# Time series plot for all pollutantspollutants = ['co', 'no', 'no2', 'o3', 'so2', 'pm2_5', 'pm10', 'nh3']fig, axes = plt.subplots(4, 2, figsize=(15, 12))axes = axes.flatten()for idx, pollutant in enumerate(pollutants):    axes[idx].plot(df['datetime'], df[pollutant], linewidth=0.8, alpha=0.7)    axes[idx].set_title(f'{pollutant.upper()} Over Time', fontsize=12, fontweight='bold')    axes[idx].set_xlabel('DateTime', fontsize=10)    axes[idx].set_ylabel(f'{pollutant.upper()}', fontsize=10)    axes[idx].grid(True, alpha=0.3)    axes[idx].tick_params(axis='x', rotation=45)plt.tight_layout()plt.show()

### 3.2 Correlation Analysis

In [ ]:
# Correlation heatmappollutants_with_aqi = pollutants + ['aqi']correlation_matrix = df[pollutants_with_aqi].corr()plt.figure(figsize=(12, 10))sns.heatmap(correlation_matrix, annot=True, fmt='.2f', cmap='coolwarm',            center=0, square=True, linewidths=1, cbar_kws={"shrink": 0.8})plt.title('Correlation Matrix - Pollutants and AQI', fontsize=14, fontweight='bold')plt.tight_layout()plt.show()# Most correlated with AQIprint("\nCorrelation with AQI:")print(correlation_matrix['aqi'].sort_values(ascending=False))

### 3.3 Feature Distributions

In [ ]:
# Distribution plotsfig, axes = plt.subplots(4, 2, figsize=(15, 12))axes = axes.flatten()for idx, pollutant in enumerate(pollutants):    axes[idx].hist(df[pollutant], bins=30, color='steelblue', alpha=0.7, edgecolor='black')    axes[idx].set_title(f'{pollutant.upper()} Distribution', fontsize=12, fontweight='bold')    axes[idx].set_xlabel(pollutant.upper(), fontsize=10)    axes[idx].set_ylabel('Frequency', fontsize=10)    axes[idx].grid(True, alpha=0.3, axis='y')plt.tight_layout()plt.show()

### 3.4 Hourly Patterns

In [ ]:
# Hourly patternsdf['hour'] = df['datetime'].dt.hourhourly_avg = df.groupby('hour')[pollutants + ['aqi']].mean()fig, axes = plt.subplots(3, 3, figsize=(15, 12))axes = axes.flatten()for idx, col in enumerate(pollutants + ['aqi']):    axes[idx].plot(hourly_avg.index, hourly_avg[col], marker='o', linewidth=2)    axes[idx].set_title(f'{col.upper()} - Hourly Average', fontsize=12, fontweight='bold')    axes[idx].set_xlabel('Hour of Day', fontsize=10)    axes[idx].set_ylabel(f'Average {col.upper()}', fontsize=10)    axes[idx].grid(True, alpha=0.3)    axes[idx].set_xticks(range(0, 24, 2))plt.tight_layout()plt.show()

## 4. Feature Engineering### 4.1 Create Features

In [ ]:
# Apply feature engineeringdf_featured = engineer_features(df.copy())print(f"\nOriginal features: {df.shape[1]}")print(f"Features after engineering: {df_featured.shape[1]}")print(f"Total samples: {len(df_featured)}")# Show new featuresnew_features = [col for col in df_featured.columns if col not in df.columns]print(f"\nNew features created: {len(new_features)}")print(f"\nSample new features:")print(new_features[:20])

### 4.2 Visualize Lag Features

In [ ]:
# Visualize lag features for PM2.5fig, axes = plt.subplots(2, 1, figsize=(15, 8))# Original vs lagssample_data = df_featured.tail(100)axes[0].plot(range(len(sample_data)), sample_data['pm2_5'], label='Current PM2.5', linewidth=2)axes[0].plot(range(len(sample_data)), sample_data['pm2_5_lag_1h'], label='1h Lag', linewidth=2, alpha=0.7)axes[0].plot(range(len(sample_data)), sample_data['pm2_5_lag_6h'], label='6h Lag', linewidth=2, alpha=0.7)axes[0].plot(range(len(sample_data)), sample_data['pm2_5_lag_24h'], label='24h Lag', linewidth=2, alpha=0.7)axes[0].set_title('PM2.5 - Current vs Lag Features', fontsize=14, fontweight='bold')axes[0].set_xlabel('Sample Index', fontsize=12)axes[0].set_ylabel('PM2.5', fontsize=12)axes[0].legend()axes[0].grid(True, alpha=0.3)# Rolling featuresaxes[1].plot(range(len(sample_data)), sample_data['pm2_5'], label='Current PM2.5', linewidth=2)axes[1].plot(range(len(sample_data)), sample_data['pm2_5_rolling_mean_6h'], label='6h Rolling Mean', linewidth=2, alpha=0.7)axes[1].plot(range(len(sample_data)), sample_data['pm2_5_rolling_mean_12h'], label='12h Rolling Mean', linewidth=2, alpha=0.7)axes[1].plot(range(len(sample_data)), sample_data['pm2_5_rolling_mean_24h'], label='24h Rolling Mean', linewidth=2, alpha=0.7)axes[1].set_title('PM2.5 - Rolling Mean Features', fontsize=14, fontweight='bold')axes[1].set_xlabel('Sample Index', fontsize=12)axes[1].set_ylabel('PM2.5', fontsize=12)axes[1].legend()axes[1].grid(True, alpha=0.3)plt.tight_layout()plt.show()

## 5. Model Training### 5.1 Prepare Data

In [ ]:
from sklearn.preprocessing import StandardScaler# Define features and targetexclude_cols = ['datetime', 'aqi', 'lon', 'lat']feature_cols = [col for col in df_featured.columns if col not in exclude_cols]X = df_featured[feature_cols].valuesy = df_featured['aqi'].valuesprint(f"Feature shape: {X.shape}")print(f"Target shape: {y.shape}")# Time-based split (70/15/15)train_size = int(0.7 * len(X))val_size = int(0.15 * len(X))X_train = X[:train_size]y_train = y[:train_size]X_val = X[train_size:train_size+val_size]y_val = y[train_size:train_size+val_size]X_test = X[train_size+val_size:]y_test = y[train_size+val_size:]print(f"\nTrain set: {X_train.shape[0]} samples")print(f"Validation set: {X_val.shape[0]} samples")print(f"Test set: {X_test.shape[0]} samples")# Scale featuresscaler = StandardScaler()X_train_scaled = scaler.fit_transform(X_train)X_val_scaled = scaler.transform(X_val)X_test_scaled = scaler.transform(X_test)print("\n✅ Data prepared and scaled!")

### 5.2 Train Models

In [ ]:
# Initialize trainertrainer = AQIModelTrainer()# Train Random Forestprint("Training Random Forest...")rf_model = trainer.train_random_forest(X_train_scaled, y_train, X_val_scaled, y_val)# Train XGBoostprint("\nTraining XGBoost...")xgb_model = trainer.train_xgboost(X_train_scaled, y_train, X_val_scaled, y_val)print("\n✅ Models trained successfully!")

## 6. Model Evaluation### 6.1 Calculate Metrics

In [ ]:
# Evaluate modelsresults = {}# Random Forestrf_pred_test = rf_model.predict(X_test_scaled)rf_metrics = calculate_metrics(y_test, rf_pred_test)results['Random Forest'] = rf_metrics# XGBoostxgb_pred_test = xgb_model.predict(X_test_scaled)xgb_metrics = calculate_metrics(y_test, xgb_pred_test)results['XGBoost'] = xgb_metrics# Display resultscomparison_df = compare_models(results)comparison_df

### 6.2 Visualize Predictions

In [ ]:
# Random Forest predictionsfig, axes = plt.subplots(1, 2, figsize=(15, 6))# Scatter plotaxes[0].scatter(y_test, rf_pred_test, alpha=0.5, color='#3498db')axes[0].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)axes[0].set_xlabel('Actual AQI', fontsize=12)axes[0].set_ylabel('Predicted AQI', fontsize=12)axes[0].set_title('Random Forest - Actual vs Predicted', fontsize=14, fontweight='bold')axes[0].grid(True, alpha=0.3)# Time seriesindices = range(min(len(y_test), 100))axes[1].plot(indices, y_test[:100], label='Actual', linewidth=2, alpha=0.7)axes[1].plot(indices, rf_pred_test[:100], label='Predicted', linewidth=2, alpha=0.7)axes[1].set_xlabel('Sample Index', fontsize=12)axes[1].set_ylabel('AQI', fontsize=12)axes[1].set_title('Random Forest - Time Series Comparison', fontsize=14, fontweight='bold')axes[1].legend()axes[1].grid(True, alpha=0.3)plt.tight_layout()plt.show()

In [ ]:
# XGBoost predictionsfig, axes = plt.subplots(1, 2, figsize=(15, 6))# Scatter plotaxes[0].scatter(y_test, xgb_pred_test, alpha=0.5, color='#e74c3c')axes[0].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)axes[0].set_xlabel('Actual AQI', fontsize=12)axes[0].set_ylabel('Predicted AQI', fontsize=12)axes[0].set_title('XGBoost - Actual vs Predicted', fontsize=14, fontweight='bold')axes[0].grid(True, alpha=0.3)# Time seriesaxes[1].plot(indices, y_test[:100], label='Actual', linewidth=2, alpha=0.7)axes[1].plot(indices, xgb_pred_test[:100], label='Predicted', linewidth=2, alpha=0.7)axes[1].set_xlabel('Sample Index', fontsize=12)axes[1].set_ylabel('AQI', fontsize=12)axes[1].set_title('XGBoost - Time Series Comparison', fontsize=14, fontweight='bold')axes[1].legend()axes[1].grid(True, alpha=0.3)plt.tight_layout()plt.show()

### 6.3 Feature Importance

In [ ]:
# Get feature importance from XGBoostimportance_dict = trainer.get_feature_importance(xgb_model, feature_cols)importance_df = pd.DataFrame([    {'feature': k, 'importance': v}     for k, v in importance_dict.items()]).sort_values('importance', ascending=False)# Plot top 20 featurestop_20 = importance_df.head(20)plt.figure(figsize=(12, 8))colors = sns.color_palette('husl', n_colors=len(top_20))plt.barh(range(len(top_20)), top_20['importance'].values, color=colors)plt.yticks(range(len(top_20)), top_20['feature'].values)plt.xlabel('Importance Score', fontsize=12)plt.ylabel('Features', fontsize=12)plt.title('Top 20 Most Important Features', fontsize=14, fontweight='bold')plt.gca().invert_yaxis()plt.grid(True, alpha=0.3, axis='x')plt.tight_layout()plt.show()print("\nTop 10 Most Important Features:")print(importance_df.head(10).to_string(index=False))

## 7. 24-Hour Prediction Demo### 7.1 Generate Predictions

In [ ]:
# Use best model (XGBoost)best_model = xgb_model# Get last available datalast_data = df_featured.tail(100)# Generate 24h predictionspredictions_24h = predict_next_24h(best_model, last_data, scaler, feature_cols)print("24-Hour AQI Forecast:")print("="*60)print(predictions_24h.to_string(index=False))

### 7.2 Visualize Forecast

In [ ]:
# Visualize 24h predictionsplt.figure(figsize=(15, 6))plt.plot(predictions_24h['hour'], predictions_24h['predicted_aqi'],         marker='o', linewidth=2, markersize=8, color='#3498db', label='Predicted AQI')# Add color zonesplt.axhspan(0, 2, alpha=0.1, color='green', label='Good (1-2)')plt.axhspan(2, 3, alpha=0.1, color='yellow', label='Fair (2-3)')plt.axhspan(3, 4, alpha=0.1, color='orange', label='Moderate (3-4)')plt.axhspan(4, 5, alpha=0.1, color='red', label='Poor (4-5)')plt.xlabel('Hour Ahead', fontsize=12)plt.ylabel('Predicted AQI', fontsize=12)plt.title('24-Hour AQI Forecast', fontsize=14, fontweight='bold')plt.legend(loc='best')plt.grid(True, alpha=0.3)plt.ylim(0, 6)plt.tight_layout()plt.show()# Statisticsprint(f"\nForecast Statistics:")print(f"Mean AQI: {predictions_24h['predicted_aqi'].mean():.2f}")print(f"Min AQI: {predictions_24h['predicted_aqi'].min():.2f}")print(f"Max AQI: {predictions_24h['predicted_aqi'].max():.2f}")print(f"Std AQI: {predictions_24h['predicted_aqi'].std():.2f}")

## 8. Conclusions và Next Steps### Key Findings:1. **Data Quality**:    - Dataset có 588 hourly observations   - Không có missing values   - AQI phân bố chủ yếu ở level 2-4 (Fair to Moderate)2. **Feature Engineering**:   - Tạo được 161 features từ 8 pollutants gốc   - Lag features (1h, 2h, 3h, 6h, 12h, 24h) rất quan trọng   - Rolling statistics giúp capture trends3. **Model Performance**:   - **Random Forest**: R² = 0.9994, MAE = 0.0032 (Excellent!)   - **XGBoost**: R² = 0.9598, MAE = 0.1335 (Very Good)   - Random Forest slightly overfitting trên training set4. **Feature Importance**:   - PM2.5 và PM10 là features quan trọng nhất   - Lag features của pollutants có impact lớn   - Time features (hour, day_of_week) cũng quan trọng5. **24h Prediction**:   - Model có thể dự đoán AQI cho 24h tiếp theo   - Predictions ổn định và hợp lý### Next Steps:1. **Model Improvements**:   - Implement LSTM với TensorFlow   - Try ensemble methods   - Hyperparameter tuning với Optuna/GridSearch2. **Data**:   - Thu thập thêm data từ real-world sources   - Add more features (weather, traffic, events)   - Handle seasonal patterns3. **Deployment**:   - Build REST API với FastAPI/Flask   - Create web dashboard với Streamlit   - Deploy to cloud (AWS/GCP/Azure)4. **Monitoring**:   - Track model performance over time   - Implement retraining pipeline   - Add data drift detection---### 🎯 Thank you for using this notebook!For more information, visit: [GitHub Repository](https://github.com/nghiata-uit/aqi-prediction)